In [1]:
# Check if the easydiffraction library is installed.
# If not, install it including the 'visualization' extras.
# This is needed, e.g., when running this as a notebook via Google Colab or
# Jupyter Notebook in an environment where the library is not pre-installed.
import builtins
import importlib.util

if hasattr(builtins, '__IPYTHON__'):
    if importlib.util.find_spec('easydiffraction') is None:
        !pip install 'easydiffraction[visualization]'


# Pair Distribution Function: NaCl, XRD

This example demonstrates a pair distribution function (PDF) analysis of
NaCl, based on data collected from an X-ray powder diffraction experiment.

The dataset is taken from:
https://github.com/diffpy/add2019-diffpy-cmi/tree/master

## Import Library

In [2]:
import easydiffraction as ed

## Create Project

In [3]:
project = ed.Project()

## Set Plotting Engine

In [4]:
project.plotter.engine = 'plotly'
project.plotter.x_min = 2.0
project.plotter.x_max = 30.0


Current plotter changed to
plotly


## Add Sample Model

In [5]:
project.sample_models.add(name='nacl')

In [6]:
project.sample_models['nacl'].space_group.name_h_m = 'F m -3 m'
project.sample_models['nacl'].space_group.it_coordinate_system_code = '1'
project.sample_models['nacl'].cell.length_a = 5.62
project.sample_models['nacl'].atom_sites.add(
    label='Na', type_symbol='Na', fract_x=0, fract_y=0, fract_z=0, wyckoff_letter='a', b_iso=1.0
)
project.sample_models['nacl'].atom_sites.add(
    label='Cl', type_symbol='Cl', fract_x=0.5, fract_y=0.5, fract_z=0.5, wyckoff_letter='b', b_iso=1.0
)

## Add Experiment

In [7]:
ed.download_from_repository('NaCl.gr', destination='data')


⚠️ Warning
File 'data/NaCl.gr' already exists and will not be overwritten.


In [8]:
project.experiments.add(
    name='xray_pdf',
    sample_form='powder',
    beam_mode='constant wavelength',
    radiation_probe='xray',
    scattering_type='total',
    data_path='data/NaCl.gr',
)


Loading measured data from ASCII file
/Users/runner/work/diffraction-lib/diffraction-lib/tutorials/data/NaCl.gr

Data loaded successfully
Experiment 🔬 'xray_pdf'. Number of data points: 5000


In [9]:
project.experiments['xray_pdf'].show_supported_peak_profile_types()


Supported peak profile types


Peak profile type,Description
gaussian-damped-sinc,Gaussian-damped sinc PDF profile


In [10]:
project.experiments['xray_pdf'].show_current_peak_profile_type()


Current peak profile type
gaussian-damped-sinc


In [11]:
project.experiments['xray_pdf'].peak_profile_type = 'gaussian-damped-sinc'


Peak profile type for experiment 'xray_pdf' changed to
gaussian-damped-sinc


In [12]:
project.experiments['xray_pdf'].peak.damp_q = 0.03
project.experiments['xray_pdf'].peak.broad_q = 0
project.experiments['xray_pdf'].peak.cutoff_q = 21
project.experiments['xray_pdf'].peak.sharp_delta_1 = 0
project.experiments['xray_pdf'].peak.sharp_delta_2 = 5
project.experiments['xray_pdf'].peak.damp_particle_diameter = 0

In [13]:
project.experiments['xray_pdf'].linked_phases.add(id='nacl', scale=0.5)

## Select Fitting Parameters

In [14]:
project.sample_models['nacl'].cell.length_a.free = True
project.sample_models['nacl'].atom_sites['Na'].b_iso.free = True
project.sample_models['nacl'].atom_sites['Cl'].b_iso.free = True

In [15]:
project.experiments['xray_pdf'].linked_phases['nacl'].scale.free = True
project.experiments['xray_pdf'].peak.damp_q.free = True
project.experiments['xray_pdf'].peak.sharp_delta_2.free = True

## Run Fitting

In [16]:
project.analysis.current_calculator = 'pdffit'
project.analysis.fit()


Current calculator changed to
pdffit

Using experiment 🔬 'xray_pdf' for 'single' fitting
🚀 Starting fit process with 'lmfit (leastsq)'...
📈 Goodness-of-fit (reduced χ²) change:


iteration,χ²,improvement [%]
1,163.09,
10,17.06,89.5% ↓
17,2.27,86.7% ↓
24,1.48,34.6% ↓
53,1.48,


🏆 Best goodness-of-fit (reduced χ²) is 1.48 at iteration 52
✅ Fitting complete.

Fit results
✅ Success: True
⏱️ Fitting time: 6.14 seconds
📏 Goodness-of-fit (reduced χ²): 1.48
📏 R-factor (Rf): 11.02%
📏 R-factor squared (Rf²): 11.38%
📏 Weighted R-factor (wR): 11.38%
📈 Fitted parameters:


,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,nacl,atom_sites,Na,b_iso,1.0000,1.1053,0.0077,Å²,10.53 % ↑
2,nacl,atom_sites,Cl,b_iso,1.0000,0.5707,0.0028,Å²,42.93 % ↓
3,nacl,cell,,length_a,5.6200,5.6018,0.0001,Å,0.32 % ↓
4,xray_pdf,linked_phases,nacl,scale,0.5000,0.4254,0.0006,,14.93 % ↓
5,xray_pdf,peak,,damp_q,0.0300,0.0606,0.0001,Å⁻¹,102.13 % ↑
6,xray_pdf,peak,,sharp_delta_2,5.0000,3.5041,0.0667,Å²,29.92 % ↓


## Plot Measured vs Calculated

In [17]:
project.plot_meas_vs_calc(expt_name='xray_pdf')